# Question 5a with feature scaling 

In [33]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
import scipy.spatial
from scipy.spatial.distance import euclidean
from mpl_toolkits import mplot3d
%matplotlib inline
import matplotlib.pyplot as plt
import statistics
import sys
import math
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

Data = pd.read_csv("BSOM_DataSet_for_HW2.csv")

In [122]:
X2 = Data.loc[:,['all_mcqs_avg_n20','CBSE_01']].values
X3 = Data.loc[:,['all_mcqs_avg_n20','CBSE_02']].values
X4 = Data.loc[:,['all_NBME_avg_n4','CBSE_01']].values
X5 = Data.loc[:,['all_NBME_avg_n4','CBSE_02']].values
Xmcqs = Data.loc[:,['all_mcqs_avg_n20']].values
Xnbme = Data.loc[:,['all_NBME_avg_n4']].values
Xcbse1 = Data.loc[:,['CBSE_01']].values
Xcbse2 = Data.loc[:,['CBSE_02']].values



y= Data.loc[:,['LEVEL']].values
train_y, test_y = y[:74,:], y[74:,:]

train_X2, test_X2 = X2[:74,:], X2[74:,:]
train_X3, test_X3 = X3[:74,:], X3[74:,:]
train_X4, test_X4 = X4[:74,:], X4[74:,:]
train_X5, test_X5 = X5[:74,:], X5[74:,:]
train_N, train_X2_Features = train_X2.shape
train_N, train_X3_Features = train_X3.shape
train_N, train_X4_Features = train_X4.shape
train_N, train_X5_Features = train_X5.shape

In [140]:
def pearsoncorr(X, Y):
    covar = np.mean(Y*X)-(np.mean(Y)*np.mean(X))
    stddev_1 = np.sqrt(np.mean(Y**2)-(np.mean(Y))**2)
    stddev_2 = np.sqrt(np.mean(X**2)-np.mean(X)**2)
    corr_coeff = covar/(stddev_1*stddev_2)
    print("Pearson Correlation Coefficient:",corr_coeff)

In [141]:
print("B/w mcqs and nbme:")
pearsoncorr(Xmcqs,Xnbme)
print("B/w mcqs and cbse1")
pearsoncorr(Xmcqs,Xcbse1)
print("B/w mcqs and cbse2")
pearsoncorr(Xmcqs,Xcbse2)
print("B/w nbme and cbse1")
pearsoncorr(Xnbme,Xcbse1)
print("B/w nbme and cbse1")
pearsoncorr(Xnbme,Xcbse2)
#pearsoncorr(Xnbme,Xcbse2)

B/w mcqs and nbme:
Pearson Correlation Coefficient: 0.9022750341290375
B/w mcqs and cbse1
Pearson Correlation Coefficient: 0.6219946085720189
B/w mcqs and cbse2
Pearson Correlation Coefficient: 0.7368048779708636
B/w nbme and cbse1
Pearson Correlation Coefficient: 0.6954437212893951
B/w nbme and cbse1
Pearson Correlation Coefficient: 0.8134564483193205


In [123]:
def sigmoid(z):
    g_of_z= 1/(1+np.exp(-z))
    return g_of_z

def iteration(y_temp,DataXX,theta):
    eta = 0.15
    prev_theta = 0
    for j in range(10000):
        output = DataXX.dot(theta)
        h = sigmoid(output)
        errors =  sigmoid(output) - y_temp
        cost_Fn = (-1/train_N)*np.sum((y_temp*np.log(h))+((1-y_temp)*np.log(1-h)))
        prev_theta = theta
        derivative = (1/train_N)* DataXX.T.dot(errors)
        theta  = theta - (eta * derivative)
        if(np.allclose(prev_theta,theta)):
            print("break theta:",theta)
            break
    #hypothesis = sigmoid(DataXX.dot(theta))
    return theta

In [124]:
#mean normalization
mean_norm_X = (train_X5-np.mean(train_X5))/(np.max(train_X5)-np.min(train_X5))

#minmax_norm_X = (train_X5-np.min(train_X5))/(np.max(train_X5)-np.min(train_X5))
DataXX = np.insert(mean_norm_X,0,1,axis=1)
print("data:",DataXX.shape)
#np.random.seed(233)
ttheta =np.random.randint(100,size= (train_X5_Features+1, 1))

#A and not B,C,D
y = Data.loc[:73,['LEVEL']]
level = {'A': 1,'B': 0,'C':0,'D':0} 
y_copy = [level[item] for item in y.LEVEL]
y_copy = np.asarray(y_copy)
y_temp = y_copy[:,None]
#print("y_copy 1:",y_temp)
#print("y_copy shape:",y_temp.shape)
theta_ovrA= iteration(y_temp,DataXX,ttheta)
hA=sigmoid(DataXX.dot(theta_ovrA))
#print("hypothesis",hA)

#B and not A,C,D
y = Data.loc[:73,['LEVEL']]
level = {'A': 0,'B': 1,'C':0,'D':0} 
y_copy = [level[item] for item in y.LEVEL]
y_copy = np.asarray(y_copy)
y_temp = y_copy[:,None]
theta_ovrB=  iteration(y_temp,DataXX,ttheta)
hB=sigmoid(DataXX.dot(theta_ovrB))
#print("hypoB:",hB)

#C and not A,B,D
y = Data.loc[:73,['LEVEL']]
level = {'A': 0,'B': 0,'C':1,'D':0} 
y_copy = [level[item] for item in y.LEVEL]
y_copy = np.asarray(y_copy)
y_temp = y_copy[:,None]
theta_ovrC=  iteration(y_temp,DataXX,ttheta)
hC=sigmoid(DataXX.dot(theta_ovrC))
#print("hypoC:",hC)

#D and not A,B,C
y = Data.loc[:73,['LEVEL']]
level = {'A': 0,'B': 0,'C':0,'D':1} 
y_copy= [level[item] for item in y.LEVEL]
y_copy = np.asarray(y_copy)
y_temp = y_copy[:,None]
theta_ovrD=  iteration(y_temp,DataXX,ttheta)
hD=sigmoid(DataXX.dot(theta_ovrD))
#print("hypoD:",hD)

concat_hypo  = np.hstack([hA, hB, hC,hD])
#print("hypo final:",concat_hypo )
#Return index of max values
hypo_final = np.argmax(concat_hypo,axis =1)
print("hypo final:",hypo_final )

mean_norm_testX = (test_X5-np.mean(test_X5))/(np.max(test_X5)-np.min(test_X5))
#minmax_norm_X = (test_X5-np.min(test_X5))/(np.max(test_X5)-np.min(test_X5))
testDataXX = np.insert(mean_norm_testX,0,1,axis=1)
test_hA=sigmoid(testDataXX.dot(theta_ovrA))
test_hB=sigmoid(testDataXX.dot(theta_ovrB))
test_hC=sigmoid(testDataXX.dot(theta_ovrC))
test_hD=sigmoid(testDataXX.dot(theta_ovrD))
tconcat_hypo  = np.hstack([test_hA, test_hB, test_hC,test_hD])

#Return index of max values
y_pred = np.argmax(tconcat_hypo,axis =1)
print("hypo final test:",y_pred )

data: (74, 3)
break theta: [[ 3.20008194]
 [62.57461374]
 [47.94752177]]


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in multiply
  if sys.path[0] == '':


hypo final: [1 2 0 0 1 2 2 0 2 1 2 1 0 1 1 2 2 1 2 0 1 0 1 0 0 1 1 0 2 2 1 2 0 0 1 2 0
 1 1 0 2 1 2 0 2 0 1 2 2 1 2 1 0 1 2 2 0 0 1 0 1 0 1 2 2 0 0 1 2 0 0 0 0 1]
hypo final test: [1 0 2 1 1 2 2 2 0 2 1 2 2 2 2 2 1 1 1 2 0 0 2 0 1 1 0 0 1 1 2 2 2 2 2 0 2
 1]


# Question 5b with feature scaling metrics

In [125]:
y = Data.loc[:,['LEVEL']]
level = {'A': 0,'B': 1,'C':2,'D':3} 
y_copy = [level[item] for item in y.LEVEL]
y_copy = np.asarray(y_copy)
y_train = y_copy[:74,None]
y_test =  y_copy[74:,None]

In [126]:
def confu_mat(y_test, y_pred):
    print(confusion_matrix(y_test, y_pred))

def recall(y_test, y_pred):
    recall_score(y_test, y_pred, average='macro')  
def precision(y_test, y_pred):
    precision_score(y_test, y_pred, average='macro') 
    
def fscore(y_true, y_pred):
    f1_macro = f1_score(y_test, y_pred, average='macro')  
    print("F1 macro :" ,f1_macro)
    f1_micro  = f1_score(y_test, y_pred, average='micro')  
    print("F1 micro :" ,f1_micro)
    f1_weighted = f1_score(y_test, y_pred, average='weighted')  
    print("F1 weighted :" ,f1_weighted)
    f1 = f1_score(y_test, y_pred, average=None)
    print("F1 score :" ,f1)      
          

In [119]:
confu_mat(y_test, y_pred)
precision(y_test, y_pred)
recall(y_test, y_pred)
fscore(y_test, y_pred)

[[ 7  0  0  0]
 [11  2  7  0]
 [ 4  3  2  0]
 [ 2  0  0  0]]
F1 macro : 0.20845878136200716
F1 micro : 0.2894736842105263
F1 weighted : 0.2200339558573854
F1 score : [0.4516129  0.16       0.22222222 0.        ]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [127]:
print(classification_report(y_test, y_pred, target_names=['A', 'B', 'C','D']))
print("Accuracy:",accuracy_score(y_test, y_pred))

             precision    recall  f1-score   support

          A       0.62      0.71      0.67         7
          B       0.75      0.45      0.56        20
          C       0.44      0.89      0.59         9
          D       0.00      0.00      0.00         2

avg / total       0.62      0.58      0.56        38

Accuracy: 0.5789473684210527


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Question 5a with Feature scaling and Regularization 

In [138]:
def sigmoid(z):
    g_of_z= 1/(1+np.exp(-z))
    return g_of_z

def iterationr(y_temp,DataXX,theta,lamda):
    eta = 0.01
    prev_theta = 0
    
    for j in range(10000):
        output = DataXX.dot(theta)
        h = sigmoid(output)
        
        cost_Fn = (-1/train_N)*np.sum((y_temp*np.log(h))+((1-y_temp)*np.log(1-h)))+ lamda/(2*train_N) * sum(np.linalg.norm(theta,1,axis=0))
        prev_theta = theta
        derivative =  DataXX.T.dot(h-y_temp)
        theta  = theta - (eta/train_N * derivative + theta*(lamda/train_N))
        theta[0] = theta[0] - (eta/train_N)* DataXX.T.dot(h - y_temp)[0]
        
        #print("theta:",  theta[0])
        if(np.allclose(prev_theta,theta)):
            print("break theta:",theta,j)
            break
    #hypothesis = DataXX.dot(theta)
    return theta

In [139]:

#mean normalization
mean_norm_X = (train_X5-np.mean(train_X5))/(np.max(train_X5)-np.min(train_X5))

DataXX = np.insert(mean_norm_X,0,1,axis=1)
#np.random.seed(233)
ttheta =np.random.randint(100,size= (train_X_Features+1, 1))


lamda = [0.0001,0.001,0.01,0.1,0.2,1,10]
for l in lamda:
    #A and not B,C,D
    y = Data.loc[:73,['LEVEL']]
    level = {'A': 1,'B': 0,'C':0,'D':0} 
    y_copy = [level[item] for item in y.LEVEL]
    y_copy = np.asarray(y_copy)
    y_temp = y_copy[:,None]
    #print("y_copy 1:",y_temp)
    #print("y_copy shape:",y_temp.shape)
    theta_ovrA= iterationr(y_temp,DataXX,ttheta,l)
    hA=sigmoid(DataXX.dot(theta_ovrA))
    #print("hypothesis",hA)

    #B and not A,C,D
    y = Data.loc[:73,['LEVEL']]
    level = {'A': 0,'B': 1,'C':0,'D':0} 
    y_copy = [level[item] for item in y.LEVEL]
    y_copy = np.asarray(y_copy)
    y_temp = y_copy[:,None]
    theta_ovrB=  iterationr(y_temp,DataXX,ttheta,l)
    hB=sigmoid(DataXX.dot(theta_ovrB))
    #print("hypoB:",hB)

    #C and not A,B,D
    y = Data.loc[:73,['LEVEL']]
    level = {'A': 0,'B': 0,'C':1,'D':0} 
    y_copy = [level[item] for item in y.LEVEL]
    y_copy = np.asarray(y_copy)
    y_temp = y_copy[:,None]
    theta_ovrC=  iterationr(y_temp,DataXX,ttheta,l)
    hC=sigmoid(DataXX.dot(theta_ovrC))
    #print("hypoC:",hC)

    #D and not A,B,C
    y = Data.loc[:73,['LEVEL']]
    level = {'A': 0,'B': 0,'C':0,'D':1} 
    y_copy= [level[item] for item in y.LEVEL]
    y_copy = np.asarray(y_copy)
    y_temp = y_copy[:,None]
    theta_ovrD=  iterationr(y_temp,DataXX,ttheta,l)
    hD=sigmoid(DataXX.dot(theta_ovrD))
    #print("hypoD:",hD)

    concat_hypo  = np.hstack([hA, hB, hC,hD])

    #Return index of max values
    hypo_final = np.argmax(concat_hypo,axis =1)
    print("hypo final for lambda ",l,":",hypo_final)
    mean_norm_testX = (test_X5-np.mean(test_X5))/(np.max(test_X5)-np.min(test_X5))
    testDataXX = np.insert(mean_norm_testX,0,1,axis=1)
    test_hA=sigmoid(testDataXX.dot(theta_ovrA))
    test_hB=sigmoid(testDataXX.dot(theta_ovrB))
    test_hC=sigmoid(testDataXX.dot(theta_ovrC))
    test_hD=sigmoid(testDataXX.dot(theta_ovrD))
    concat_hypo  = np.hstack([test_hA, test_hB, test_hC,test_hD])

    #Return index of max values
    y_pred = np.argmax(concat_hypo,axis =1)
    print("hypo final test for lambda ",l,":",y_pred)
    confu_mat(y_test, y_pred)
    precision(y_test, y_pred)
    recall(y_test, y_pred)
    print(classification_report(y_test, y_pred, target_names=['A', 'B', 'C','D']))
    print("Accuracy:",accuracy_score(y_test, y_pred))

break theta: [[-4.02637317]
 [42.85729229]
 [46.84644326]] 4925
break theta: [[-8.95344854]
 [43.77197588]
 [44.2218354 ]] 5718
hypo final for lambda  0.0001 : [1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 0 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1]
hypo final test for lambda  0.0001 : [1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1
 1]
[[ 5  2  0  0]
 [ 1 19  0  0]
 [ 0  9  0  0]
 [ 0  2  0  0]]
             precision    recall  f1-score   support

          A       0.83      0.71      0.77         7
          B       0.59      0.95      0.73        20
          C       0.00      0.00      0.00         9
          D       0.00      0.00      0.00         2

avg / total       0.47      0.63      0.53        38

Accuracy: 0.631578947368421


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


hypo final for lambda  0.001 : [1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 0 0 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 0 1 0 1 1]
hypo final test for lambda  0.001 : [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1
 1]
[[ 3  4  0  0]
 [ 1 19  0  0]
 [ 0  9  0  0]
 [ 0  2  0  0]]
             precision    recall  f1-score   support

          A       0.75      0.43      0.55         7
          B       0.56      0.95      0.70        20
          C       0.00      0.00      0.00         9
          D       0.00      0.00      0.00         2

avg / total       0.43      0.58      0.47        38

Accuracy: 0.5789473684210527
hypo final for lambda  0.01 : [1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 1 0 0 1 1 1
 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 1 1 1 1 1 0 1 1 0 1 0 1 1]
hypo final test for lambda  0.01 : [1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 0 0 1 1 1 1 1 1 1 1 1
 